In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.datasets import cifar10
#from keras.datasets import cifar10  
(x_train_old, y_train_old), (x_test_old, y_test_old) = cifar10.load_data()

In [ ]:
print('Traning data shape:', x_train_old.shape)
print('Testing data shape:', x_test_old.shape)

y_train_old.shape,y_test_old.shape

In [ ]:
x_train = []
y_train = []
x_test = []
y_test = []


for i in range(50000):
    if (y_train_old[i]==3) or (y_train_old[i]==5):
        y_train.append(y_train_old[i])
        #x_train.append([x_train_old[i*500:(i+1)*500]])
        x_train.append(x_train_old[i])
        #print(len(x_train))
        
for i in range(10000):
    if (y_test_old[i]==3) or (y_test_old[i]==5):
        y_test.append(y_test_old[i])
        x_test.append(x_test_old[i])
        
        #print(len(x_test))


In [ ]:
x_train_new = np.array(x_train, dtype=object)
y_train_new = np.array(y_train, dtype=object)
x_train_new = x_train_new[:8000, :32, :32 , :3]
y_train_new = y_train_new[:8000, :1]

x_test_new = np.array(x_test, dtype=object)
y_test_new = np.array(y_test, dtype=object)
x_train_new.shape, y_train_new.shape,x_test_new.shape, y_test_new.shape

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

label_dict = {
 0: 'airplane',
 1: 'automobile',
 2: 'bird',
 3: 'cat',
 4: 'deer',
 5: 'dog',
 6: 'frog',
 7: 'horse',
 8: 'ship',
 9: 'truck',
}

plt.figure(figsize=[5,5])

# Display the 20th/any image in training data
plt.subplot(121)
curr_img = np.reshape(x_train[20], (32,32,3))
plt.imshow(curr_img)
print(plt.title("(Label: " + str(label_dict[y_train[20][0]]) + ")"))

# Display the sixth/any image in testing data
plt.subplot(122)
curr_img = np.reshape(x_test[6],(32,32,3))
plt.imshow(curr_img)
print(plt.title("(Label: " + str(label_dict[y_test[6][0]]) + ")"))

In [ ]:
# create Dataframe
x_train_flat = x_train_new.reshape(-1,3072)
feat_cols = ['pixel'+str(i) for i in range(x_train_flat.shape[1])]
df_cifar = pd.DataFrame(x_train_flat,columns=feat_cols)
df_cifar['label'] = y_train_new
print('Size of the dataframe: {}'.format(df_cifar.shape))


In [ ]:
from sklearn.decomposition import PCA
pca_cifar = PCA(n_components=2)
principalComponents_cifar = pca_cifar.fit_transform(df_cifar.iloc[:,:-1])

principal_cifar_Df = pd.DataFrame(data = principalComponents_cifar
             , columns = ['principal component 1', 'principal component 2'])
principal_cifar_Df['y'] = y_train_new
principal_cifar_Df.head()

In [ ]:
#variance
print('Explained variation per principal component: {}'.format(pca_cifar.explained_variance_ratio_))

In [ ]:
import seaborn as sns
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="principal component 1", y="principal component 2",
    hue="y",
    palette=sns.color_palette("hls", 2),
    data=principal_cifar_Df,
    legend="full",
    alpha=0.3
)

In [ ]:
x_test_new = x_test_new/255.0
x_test_new = x_test_new.reshape(-1,32,32,3)
x_test_flat = x_test_new.reshape(-1,3072)

#make the instance of the PCA model.
pca = PCA(0.9)
pca.fit(x_train_flat)
PCA(copy=True, iterated_power='auto', n_components=0.9, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)


In [ ]:
#dimention reduction
pca.n_components_
train_img_pca = pca.transform(x_train_flat)
test_img_pca = pca.transform(x_test_flat)
y_train = y_train[:8000]
train_img_pca.shape, test_img_pca.shape, len(y_train)


In [ ]:
#work on model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import utils as np_utils
#from tensorflow.keras.utils.np_utils import to_categorical
from tensorflow.keras.optimizers import RMSprop

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

batch_size = 128
num_classes = 6
epochs = 20

model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(87,)))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

In [ ]:
y_train.shape

In [ ]:
import tensorflow as tf
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])
#y_train = tf.reshape(y_train, [2000, 1])
#train_img_pca = tf.convert_to_tensor(train_img_pca, dtype=tf.float32)
history = model.fit(train_img_pca, y_train,batch_size=batch_size,epochs=epochs,verbose=1,
                    validation_data=(test_img_pca, y_test))

In [ ]:
print(len(train_img_pca))

In [ ]:
train_img_pca[0], train_img_pca.shape

In [ ]:
#train_img_pca.tolist()
train_img_pca = np.array(train_img_pca)
train_img_pca.resize(8000,32,32,1,refcheck=False)
y_train = np.array(y_train)
y_train.resize(8000,1,refcheck=False)
y_train.shape,train_img_pca.shape

In [ ]:
# %% [code]
import random
import cv2
IMG_SIZE = 32
#train_img_pca.resize(32,32,3)
#random.shuffle(train_img_pca)


print(train_img_pca[0].reshape(-1, IMG_SIZE, IMG_SIZE, 1))

train_img_pca = np.array(train_img_pca).reshape(-1, IMG_SIZE, IMG_SIZE, 1)


import pickle

pickle_out = open("X.pickle","wb")
pickle.dump(train_img_pca, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y_train, pickle_out)
pickle_out.close()

pickle_in = open("X.pickle","rb")
train_img_pca = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y_train = pickle.load(pickle_in)

In [ ]:
y_train.shape, train_img_pca.shape

In [ ]:
# %% [code]

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

pickle_in = open("X.pickle","rb")
train_img_pca = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y_train = pickle.load(pickle_in)

train_img_pca = train_img_pca/255.0

train_img_pca = np.array(train_img_pca).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

y_train = np.array(y_train)
dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=train_img_pca.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'],
                          )

            model.fit(train_img_pca, 
                      y_train,
                      batch_size=32,
                      epochs=25,
                      validation_split=0.3,
                      callbacks=[tensorboard])

model.save('is_Cat_or_Dog.model')



In [ ]:
import cv2
import tensorflow as tf

#print(model.summary())

class_category = ["Dog", "Cat"]


def prepare(filepath):
    IMG_SIZE = 50  # 50 in txt-based

    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    #new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)


model = tf.keras.models.load_model('is_Cat_or_Dog.model')

prediction = model.predict([prepare('D:/Image_Dataset/CatDogwithNN/test_set/test_set/cats/cat.4013.jpg')])
print(prediction,'\n')  # will be a list in a list.

print(class_category[int(prediction[0][0])])

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img_array, cmap='gray')
plt.show()